<a href="https://colab.research.google.com/github/msironvalle/WebSemanticaMTI2025/blob/main/Tarea1_vFinal_ajson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install rdflib pyshex

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.9/564.9 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.8/69.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 5.4 MB/s eta 0:00:00
  Created wheel for cfgraph: filename=CFGraph-0.2.1-py3-none-any.whl size=2214 sha256=142679b075724baf8259c100e1c8488d9dcc61907f29348c37b6ebc51b5f09a6
  Stored in directory: /root/.cache/pip/wheels/8b/ac/8f/0f5b041561f10e6db39796e4ad11011b8f5a4b0a0ed74a5e61
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144555 sha256=ee6db227899f399e24ea02303c5b769d32bbd79343dc98fabc483786083824f9
  Stored in directory: /root/.cache/pip/wheels/1a/97/32/461f837398029ad76911109f

In [ ]:
import json
import os
import decimal
from rdflib import Graph, Literal, RDF, URIRef, Namespace
from rdflib.namespace import XSD, FOAF, DC, OWL
from pyshex import ShExEvaluator

In [ ]:
# Definir la ruta al archivo .ajson en Google Drive
file_path = '/content/drive/My Drive/PasosFronterizos.ajson'

# Verificar si el archivo existe
if not os.path.exists(file_path):
    raise FileNotFoundError(f"El archivo {file_path} no se encuentra. Por favor, verifica la ruta.")


In [ ]:
# Leer y parsear el archivo .ajson
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# Verificar la estructura del JSON
if not isinstance(data, list) or len(data) < 2:
    raise ValueError("El archivo .ajson debe ser una lista de listas con al menos una fila de encabezados y una fila de datos.")

headers = data[0]  # Primera fila: encabezados
rows = data[1:]    # Filas restantes: datos

In [ ]:
# Definir los prefijos
rdf_prefixes = """
@prefix ex: <http://example.org/pasos_fronterizos#> .
@prefix geo: <http://www.w3.org/2003/01/geo/wgs84_pos#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

"""

# Definir la clase
rdf_class = "ex:PasoFronterizo a owl:Class .\n\n"

# Construir las instancias
rdf_instances = ""
for idx, row in enumerate(rows, start=1):
    # Asegurarse de que cada fila tenga exactamente 6 elementos
    if len(row) != 6:
        print(f"Advertencia: La fila {idx} no tiene 6 elementos y será omitida.")
        continue

    nombre = row[0].replace('"', '\\"')
    region = row[1].replace('"', '\\"')
    altura = row[2].replace(",", ".")
    unidad = row[3].replace('"', '\\"')
    latitud = row[4].replace(",", ".")
    longitud = row[5].replace(",", ".")

    # Crear la instancia en Turtle
    instance = f"""ex:PasoFronterizo_{idx} a ex:PasoFronterizo ;
    ex:altura "{altura}"^^xsd:integer ;
    ex:nombre "{nombre}" ;
    ex:region "{region}" ;
    ex:unidad "{unidad}" ;
    geo:lat "{latitud}"^^xsd:decimal ;
    geo:long "{longitud}"^^xsd:decimal .

"""
    rdf_instances += instance

# Combinar todo
rdf_complete = rdf_prefixes + rdf_class + rdf_instances


# Guardar el RDF en un archivo
output_path = '/content/drive/My Drive/output_PasosFronterizos_6.rdf'
with open(output_path, "w", encoding='utf-8') as f:
    f.write(rdf_complete)

print(f"Archivo RDF guardado en: {output_path}")

Archivo RDF guardado en: /content/drive/My Drive/output_PasosFronterizos_6.rdf


XML

In [ ]:
import os
import xml.etree.ElementTree as ET
import decimal
from pyshex import ShExEvaluator
from rdflib import Graph, Namespace, RDF


In [ ]:
# Definir la ruta al archivo XML en Google Drive
xml_file_path = '/content/drive/My Drive/PoblaciondeChile.xml'

# Verificar si el archivo existe
if not os.path.exists(xml_file_path):
    raise FileNotFoundError(f"El archivo {xml_file_path} no se encuentra. Por favor, verifica la ruta.")

In [ ]:
# Parsear el archivo XML
tree = ET.parse(xml_file_path)
root = tree.getroot()

# Verificar que la raíz sea <table>
if root.tag != 'table':
    raise ValueError("El archivo XML no tiene una estructura válida. Se espera que la raíz sea <table>.")

# Extraer todas las filas
rows = root.findall('row')

# Verificar que haya al menos una fila de encabezados y una fila de datos
if len(rows) < 2:
    raise ValueError("El archivo XML debe contener al menos una fila de encabezados y una fila de datos.")

# Extraer encabezados de la primera fila
header_row = rows[0]
headers = [col.text.strip() for col in header_row.findall('column')]

# Extraer datos de las filas restantes
data_rows = []
for row in rows[1:]:
    columns = [col.text.strip() for col in row.findall('column')]
    if len(columns) != len(headers):
        print(f"Advertencia: La fila con datos {columns} no tiene el mismo número de columnas que los encabezados y será omitida.")
        continue
    data_rows.append(columns)

# Mostrar las cabeceras y algunas filas de datos para verificar
print("Encabezados:", headers)
print("Primeras 5 filas de datos:")
for row in data_rows[:5]:
    print(row)

Encabezados: ['REGIÓN', 'EDAD', 'GÉNERO', 'POBLACIÓN', 'AÑO']
Primeras 5 filas de datos:
['Tarapacá', '0', 'Hombres', '2.960', '2012']
['Tarapacá', '1', 'Hombres', '2.974', '2012']
['Tarapacá', '2', 'Hombres', '2.916', '2012']
['Tarapacá', '3', 'Hombres', '2.847', '2012']
['Tarapacá', '4', 'Hombres', '2.681', '2012']


In [ ]:
# Definir Namespaces
ex_namespace = "http://example.org/poblacion#"
geo_namespace = "http://www.w3.org/2003/01/geo/wgs84_pos#"

# Definir el mapeo de encabezados a propiedades RDF
header_mapping = {
    "REGIÓN": "region",
    "EDAD": "edad",
    "GÉNERO": "genero",
    "POBLACIÓN": "poblacion",
    "AÑO": "ano"
}

# Definir los tipos de datos para cada propiedad
datatype_mapping = {
    "REGIÓN": "xsd:string",
    "EDAD": "xsd:string",
    "GÉNERO": "xsd:string",
    "POBLACIÓN": "xsd:decimal",
    "AÑO": "xsd:integer"
}

In [ ]:
# Iniciar la cadena RDF con los prefijos
rdf_prefixes = f"""
@prefix ex: <{ex_namespace}> .
@prefix geo: <{geo_namespace}> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .

"""

# Definir la clase PopulationData
rdf_class = "ex:PopulationData a owl:Class .\n\n"

# Construir las instancias
rdf_instances = ""
for idx, row in enumerate(data_rows, start=1):
    region = row[0].replace('"', '\\"')
    edad = row[1]
    genero = row[2].replace('"', '\\"')
    # Eliminar los puntos que actúan como separadores de miles
    poblacion = row[3].replace(".", "")
    ano = row[4]

    # Crear la instancia en Turtle
    instance = f"""ex:PopulationData_{idx} a ex:PopulationData ;
ex:region "{region}"^^xsd:string ;
ex:edad "{edad}"^^xsd:string ;
ex:genero "{genero}"^^xsd:string ;
ex:poblacion "{poblacion}"^^xsd:integer ;
ex:ano "{ano}"^^xsd:integer .

"""
    rdf_instances += instance

# Combinar todo
rdf_complete = rdf_prefixes + rdf_class + rdf_instances

# Mostrar una parte del RDF generado para verificar
print(rdf_complete[:1000])  # Imprime los primeros 1000 caracteres



@prefix ex: <http://example.org/poblacion#> .
@prefix geo: <http://www.w3.org/2003/01/geo/wgs84_pos#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .

ex:PopulationData a owl:Class .

ex:PopulationData_1 a ex:PopulationData ;
ex:region "Tarapacá"^^xsd:string ;
ex:edad "0"^^xsd:string ;
ex:genero "Hombres"^^xsd:string ;
ex:poblacion "2960"^^xsd:integer ;
ex:ano "2012"^^xsd:integer .

ex:PopulationData_2 a ex:PopulationData ;
ex:region "Tarapacá"^^xsd:string ;
ex:edad "1"^^xsd:string ;
ex:genero "Hombres"^^xsd:string ;
ex:poblacion "2974"^^xsd:integer ;
ex:ano "2012"^^xsd:integer .

ex:PopulationData_3 a ex:PopulationData ;
ex:region "Tarapacá"^^xsd:string ;
ex:edad "2"^^xsd:string ;
ex:genero "Hombres"^^xsd:string ;
ex:poblacion "2916"^^xsd:integer ;
ex:ano "2012"^^xsd:integer .

ex:PopulationData_4 a ex:PopulationData ;
ex:region "Tarapacá"^^xsd:string ;
ex:edad "3"^^xsd:string ;
ex:genero "Hombres"^^xsd:string ;
ex:poblacion "2847"

In [ ]:
# Definir la ruta para guardar el archivo RDF en Drive
output_rdf_path = '/content/drive/My Drive/output_Poblacion_2.rdf'

# Guardar el archivo RDF en modo de texto
with open(output_rdf_path, "w", encoding='utf-8') as f:
    f.write(rdf_complete)

print(f"Archivo RDF guardado en: {output_rdf_path}")

Archivo RDF guardado en: /content/drive/My Drive/output_Poblacion_2.rdf


In [ ]:
# Imprimir una parte del RDF generado para verificar
print(rdf_complete[:1000])  # Imprime los primeros 1000 caracteres


@prefix ex: <http://example.org/poblacion#> .
@prefix geo: <http://www.w3.org/2003/01/geo/wgs84_pos#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .

ex:PopulationData a owl:Class .

ex:PopulationData_1 a ex:PopulationData ;
ex:region "Tarapacá"^^xsd:string ;
ex:edad "0"^^xsd:string ;
ex:genero "Hombres"^^xsd:string ;
ex:poblacion "2960"^^xsd:integer ;
ex:ano "2012"^^xsd:integer .

ex:PopulationData_2 a ex:PopulationData ;
ex:region "Tarapacá"^^xsd:string ;
ex:edad "1"^^xsd:string ;
ex:genero "Hombres"^^xsd:string ;
ex:poblacion "2974"^^xsd:integer ;
ex:ano "2012"^^xsd:integer .

ex:PopulationData_3 a ex:PopulationData ;
ex:region "Tarapacá"^^xsd:string ;
ex:edad "2"^^xsd:string ;
ex:genero "Hombres"^^xsd:string ;
ex:poblacion "2916"^^xsd:integer ;
ex:ano "2012"^^xsd:integer .

ex:PopulationData_4 a ex:PopulationData ;
ex:region "Tarapacá"^^xsd:string ;
ex:edad "3"^^xsd:string ;
ex:genero "Hombres"^^xsd:string ;
ex:poblacion "2847"

In [ ]:
!pip install --upgrade pyshex

In [ ]:

# Definir la ruta al esquema ShEx
shex_path = '/content/drive/My Drive/schemaPoblacion_2.shex'


# Verificar si el archivo ShEx existe
if not os.path.exists(shex_path):
    raise FileNotFoundError(f"El esquema ShEx {shex_path} no se encuentra. Por favor, verifica la ruta.")

# Leer RDF y ShEx
with open(output_rdf_path, "r", encoding="utf-8") as rdf_file:
    rdf_data = rdf_file.read()

with open(shex_path, "r", encoding="utf-8") as shex_file:
    shex_schema = shex_file.read()

# Inicializar el evaluador sin validate_shapes
evaluator = ShExEvaluator(rdf=rdf_data, schema=shex_schema)

# Realizar la evaluación
results = evaluator.evaluate()

# Imprimir resultados
for result in results:
    conformidad = "Sí" if result.result else "No"
    print(f"Conformidad: {conformidad}, Nodo: {result.focus}")
    if result.message:
        print(f"Mensaje: {result.message}\n")

Conformidad: No, Nodo: http://example.org/poblacion#PopulationData


AttributeError: 'EvaluationResult' object has no attribute 'message'

In [ ]:
from rdflib import Graph, Namespace, RDF
from pyshex import ShExEvaluator

# Definir el Namespace
ex = Namespace("http://example.org/poblacion#")

# Cargar el RDF
g = Graph()
g.parse(output_rdf_path, format='turtle')

# Encontrar todas las instancias de ex:PopulationData
instances = list(g.subjects(RDF.type, ex.PopulationData))

# Excluir la clase 'PopulationData' si está incluida accidentalmente
instances = [inst for inst in instances if inst != ex.PopulationData]

print(f"Total de instancias encontradas: {len(instances)}")
print("Primeras 5 instancias:")
for inst in instances[:5]:
    print(inst)

# Leer el esquema ShEx
with open(shex_path, "r", encoding="utf-8") as shex_file:
    shex_schema = shex_file.read()

# Leer el RDF completo como string para pyshex
with open(output_rdf_path, "r", encoding="utf-8") as rdf_file:
    rdf_data = rdf_file.read()

# Inicializar el evaluador ShEx
evaluator = ShExEvaluator(rdf=rdf_data, schema=shex_schema)

# Validar un subconjunto de instancias inicialmente (por ejemplo, las primeras 10)
for instance in instances[:10]:
    instance_str = str(instance)  # Convertir a string para pyshex
    # Realizar la evaluación de la instancia contra el shape PopulationDataShape
    try:
        results = evaluator.evaluate(focus_node=instance_str, shapes="http://example.org/poblacion#PopulationDataShape")

        for result in results:
            conformidad = "Sí" if result.result else "No"
            print(f"Conformidad: {conformidad}, Nodo: {result.focus}")
            if result.explanation:
                print(f"Mensaje: {result.explanation}\n")
    except TypeError as e:
        print(f"Error al evaluar el nodo {instance_str}: {e}")



Total de instancias encontradas: 7776
Primeras 5 instancias:
http://example.org/poblacion#PopulationData_1
http://example.org/poblacion#PopulationData_2
http://example.org/poblacion#PopulationData_3
http://example.org/poblacion#PopulationData_4
http://example.org/poblacion#PopulationData_5
Error al evaluar el nodo http://example.org/poblacion#PopulationData_1: ShExEvaluator.evaluate() got an unexpected keyword argument 'focus_node'
Error al evaluar el nodo http://example.org/poblacion#PopulationData_2: ShExEvaluator.evaluate() got an unexpected keyword argument 'focus_node'
Error al evaluar el nodo http://example.org/poblacion#PopulationData_3: ShExEvaluator.evaluate() got an unexpected keyword argument 'focus_node'
Error al evaluar el nodo http://example.org/poblacion#PopulationData_4: ShExEvaluator.evaluate() got an unexpected keyword argument 'focus_node'
Error al evaluar el nodo http://example.org/poblacion#PopulationData_5: ShExEvaluator.evaluate() got an unexpected keyword argume

# CSV: Colegios en Durham

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install --upgrade rdflib
!pip install --upgrade pyshacl
!pip install pyshex


In [ ]:
import os
import csv
from rdflib import Graph, Namespace, RDF, Literal
from rdflib.namespace import XSD, GEO, OWL
from pyshacl import validate
from pyshex import ShExEvaluator

In [ ]:
# Rutas a los archivos en Google Drive
csv_file_path = '/content/drive/My Drive/SchoolsDurham.csv'  # Reemplaza con tu ruta

# Verificar que los archivos existan
if not os.path.exists(csv_file_path):
    raise FileNotFoundError(f"El archivo CSV no se encuentra en la ruta especificada: {csv_file_path}")


In [ ]:
# Leer y procesar el archivo CSV
data_rows = []
with open(csv_file_path, 'r', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    headers = next(reader)  # Leer la fila de encabezados
    for idx, row in enumerate(reader, start=1):
        # Asegurarse de que cada fila tenga exactamente 9 elementos
        if len(row) != 9:
            print(f"Advertencia: La fila {idx} no tiene 9 elementos y será omitida.")
            continue

        nombre = row[0].strip().replace('"', '\\"')
        address1 = row[1].strip().replace('"', '\\"')
        address2 = row[2].strip().replace('"', '\\"')
        address3 = row[3].strip().replace('"', '\\"') if row[3].strip() else None  # Puede ser None
        postcode = row[4].strip().replace('"', '\\"')
        easting = row[5].strip().replace(",", "")  # Eliminar separadores de miles si los hubiera
        northing = row[6].strip().replace(",", "")
        lat = row[7].strip().replace(",", ".")
        long = row[8].strip().replace(",", ".")

        # Validar y convertir los tipos de datos
        try:
            easting_decimal = float(easting)
        except ValueError:
            print(f"Error: El easting '{easting}' en la fila {idx} no es un decimal válido.")
            continue

        try:
            northing_decimal = float(northing)
        except ValueError:
            print(f"Error: El northing '{northing}' en la fila {idx} no es un decimal válido.")
            continue

        try:
            lat_decimal = float(lat)
            if not (-90 <= lat_decimal <= 90):
                raise ValueError
        except ValueError:
            print(f"Error: La latitud '{lat}' en la fila {idx} está fuera del rango válido (-90 a 90).")
            continue

        try:
            long_decimal = float(long)
            if not (-180 <= long_decimal <= 180):
                raise ValueError
        except ValueError:
            print(f"Error: La longitud '{long}' en la fila {idx} está fuera del rango válido (-180 a 180).")
            continue

        data_rows.append({
            'nombre': nombre,
            'address1': address1,
            'address2': address2,
            'address3': address3,
            'postcode': postcode,
            'easting': easting_decimal,
            'northing': northing_decimal,
            'lat': lat_decimal,
            'long': long_decimal
        })

# Mostrar algunas filas procesadas para verificar
print("Primeras 5 filas procesadas:")
for row in data_rows[:5]:
    print(row)

Primeras 5 filas procesadas:
{'nombre': 'Aclet Close Nursery', 'address1': 'Aclet Close', 'address2': 'Bishop Auckland', 'address3': None, 'postcode': 'DL146PX', 'easting': 420567.375, 'northing': 527882.1876, 'lat': 54.645515, 'long': -1.681249}
{'nombre': 'Acre Rigg Academy', 'address1': 'Acre Rigg Road', 'address2': 'Peterlee', 'address3': None, 'postcode': 'SR8 2DU', 'easting': 441971.3751, 'northing': 541494.25, 'lat': 54.766521, 'long': -1.347589}
{'nombre': 'Acre Rigg Infant', 'address1': 'Acre Rigg Road', 'address2': 'Peterlee', 'address3': None, 'postcode': 'SR8 2DU', 'easting': 442029.7812, 'northing': 541494.8125, 'lat': 54.766521, 'long': -1.346682}
{'nombre': 'All Saints RC VA Primary', 'address1': 'Kitswell Road', 'address2': 'Lanchester', 'address3': 'Durham', 'postcode': 'DH7 0JG', 'easting': 416323.7813, 'northing': 547746.9375, 'lat': 54.824205, 'long': -1.7459}
{'nombre': 'Annfield Plain Infant', 'address1': 'Northgate', 'address2': 'Annfield Plain', 'address3': 'Sta

In [ ]:
# Definir Namespaces
EX = Namespace("http://example.org/schools#")
GEO = Namespace("http://www.w3.org/2003/01/geo/wgs84_pos#")
OWL_NS = Namespace("http://www.w3.org/2002/07/owl#")
XSD_NS = Namespace("http://www.w3.org/2001/XMLSchema#")

# Iniciar el grafo RDF
rdf_prefixes = f"""
@prefix ex: <{EX}> .
@prefix geo: <{GEO}> .
@prefix xsd: <{XSD_NS}> .
@prefix owl: <{OWL_NS}> .

"""

# Definir la clase School
rdf_class = "ex:School a owl:Class .\n\n"

# Construir las instancias
rdf_instances = ""
for idx, row in enumerate(data_rows, start=1):
    nombre = row['nombre']
    address1 = row['address1']
    address2 = row['address2']
    address3 = row['address3']
    postcode = row['postcode']
    easting = row['easting']
    northing = row['northing']
    lat = row['lat']
    long = row['long']

    # Crear la instancia en Turtle
    instance = f"""ex:School_{idx} a ex:School ;
    ex:name "{nombre}"^^xsd:string ;
    ex:address1 "{address1}"^^xsd:string ;
    ex:address2 "{address2}"^^xsd:string ;
"""
    if address3:
        instance += f"    ex:address3 \"{address3}\"^^xsd:string ;\n"
    else:
        instance += f"    ex:address3 \"\"^^xsd:string ;\n"  # Representar como cadena vacía

    instance += f"""    ex:postcode "{postcode}"^^xsd:string ;
    ex:easting "{easting}"^^xsd:decimal ;
    ex:northing "{northing}"^^xsd:decimal ;
    ex:lat "{lat}"^^xsd:decimal ;
    ex:long "{long}"^^xsd:decimal .

    """
    rdf_instances += instance

# Combinar todo
rdf_complete = rdf_prefixes + rdf_class + rdf_instances

# Mostrar una parte del RDF generado para verificar
print(rdf_complete[:1000])  # Imprime los primeros 1000 caracteres


@prefix ex: <http://example.org/schools#> .
@prefix geo: <http://www.w3.org/2003/01/geo/wgs84_pos#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .

ex:School a owl:Class .

ex:School_1 a ex:School ;
    ex:name "Aclet Close Nursery"^^xsd:string ;
    ex:address1 "Aclet Close"^^xsd:string ;
    ex:address2 "Bishop Auckland"^^xsd:string ;
    ex:address3 ""^^xsd:string ;
    ex:postcode "DL146PX"^^xsd:string ;
    ex:easting "420567.375"^^xsd:decimal ;
    ex:northing "527882.1876"^^xsd:decimal ;
    ex:lat "54.645515"^^xsd:decimal ;
    ex:long "-1.681249"^^xsd:decimal .

    ex:School_2 a ex:School ;
    ex:name "Acre Rigg Academy"^^xsd:string ;
    ex:address1 "Acre Rigg Road"^^xsd:string ;
    ex:address2 "Peterlee"^^xsd:string ;
    ex:address3 ""^^xsd:string ;
    ex:postcode "SR8 2DU"^^xsd:string ;
    ex:easting "441971.3751"^^xsd:decimal ;
    ex:northing "541494.25"^^xsd:decimal ;
    ex:lat "54.766521"^^xsd:decimal ;
    e

In [ ]:
# Definir la ruta para guardar el archivo RDF en Drive
output_rdf_path = '/content/drive/My Drive/output_SchoolsDurham.rdf'

with open(output_rdf_path, "w", encoding='utf-8') as f:
    f.write(rdf_complete)

print(f"Archivo RDF guardado en: {output_rdf_path}")

Archivo RDF guardado en: /content/drive/My Drive/output_SchoolsDurham.rdf
